# 获取Candle数据

API文档： https://www.okex.me/docs/zh/#futures-line

参数： instrument_id, start, end, granularity

In [1]:
import pandas as pd
from datetime import datetime,timedelta
import requests
import json

In [2]:
url = 'https://www.okex.me/api/futures/v3/instruments/BTC-USDT-200327/candles?'
params = {'granularity':60, 'end': datetime.utcfromtimestamp(int(datetime.timestamp(datetime.now()))).strftime('%Y-%m-%dT%H:%M:%SZ')}
r = requests.get(url, params = params,timeout=10).json()
print(pd.DataFrame(r).tail())

                            0       1       2       3       4       5        6
295  2020-01-10T22:22:00.000Z  8149.2  8150.3  8147.3  8147.3  378984  37.8984
296  2020-01-10T22:21:00.000Z  8153.2  8153.2  8145.1  8149.1  497714  49.7714
297  2020-01-10T22:20:00.000Z  8140.6  8153.2  8140.6  8153.2    3580    0.358
298  2020-01-10T22:19:00.000Z  8140.8    8142  8139.9  8140.5     204   0.0204
299  2020-01-10T22:18:00.000Z  8129.1  8143.8  8129.1  8139.7  288695  28.8695


# 循环获取K线
限制1440根

In [3]:
def make_ts(dt):
    return int(datetime.timestamp(dt))
def make_dtstr(ts):
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%dT%H:%M:%SZ')

def getCandles(instrument_id, granularity, start=None, end=None):
    params = {'granularity':granularity}
    url = 'https://www.okex.me/api/futures/v3/instruments/'+instrument_id+'/candles?'
    end_ts = make_ts(datetime.now())
    candles=[]
    for i in range(5):
        r_start=end_ts- 290 * granularity
        params["start"]= make_dtstr(r_start)
        params["end"] = make_dtstr(end_ts)
        # 访问接口数据转化为json格式
        # headers={"contentType": "application/x-www-form-urlencoded"}, 
        r = requests.get(url, params = params,timeout=10).json()
        print(params["start"],params["end"],len(r))
        candles.extend(r)
        end_ts = r_start
    # 转DataFrame与转datetime格式
    btcDf = pd.DataFrame(candles, columns=['datetime', 'open', 'high', 'low', 'close', 'volume', 'BTC_volume'])
    btcDf["datetime"] = list(map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"), btcDf["datetime"]))
    return btcDf

In [4]:
btcDf = getCandles('BTC-USDT-200327',60)

2020-01-10T22:27:36Z 2020-01-11T03:17:36Z 290
2020-01-10T17:37:36Z 2020-01-10T22:27:36Z 290
2020-01-10T12:47:36Z 2020-01-10T17:37:36Z 290
2020-01-10T07:57:36Z 2020-01-10T12:47:36Z 290
2020-01-10T03:07:36Z 2020-01-10T07:57:36Z 280


In [5]:
btcDf.tail()

,datetime,open,high,low,close,volume,BTC_volume
1435,2020-01-10 03:22:00,7921.6,7922.8,7920.6,7920.6,184921,18.4921
1436,2020-01-10 03:21:00,7920.1,7923.5,7919.5,7920.8,430546,43.0546
1437,2020-01-10 03:20:00,7911.7,7920.7,7911.7,7920.7,212480,21.248
1438,2020-01-10 03:19:00,7908.7,7910.6,7908.7,7910.6,474,0.0474
1439,2020-01-10 03:18:00,7906.8,7910.4,7906.8,7909.7,5076,0.5076


## 转成VNPY需要的数据格式

In [6]:
import pymongo

symbol = 'btc_usdt_cq.future:okex'
client = pymongo.MongoClient('localhost', 33333)

collection = client['VnTrader_1Min_Db_Coin'][symbol]
collection.create_index([('datetime', pymongo.ASCENDING)], unique=True)

def transfer2VNPY(data, symbol, collection):
    for index, row in data.iterrows():
        bar = {}
        bar['open'] = row.open
        bar['close'] = row.close
        bar['high'] = row.high
        bar['low'] = row.low
        bar['volume'] = row.volume
        bar['symbol'] = symbol
        bar['vtSymbol'] = symbol
        bar['datetime'] = row.datetime
        bar['date'] = bar['datetime'].date().strftime('%Y%m%d')
        bar['time'] = bar['datetime'].time().strftime('%H:%M:%S')
        ######################################
        flt = {'datetime': bar['datetime']}
        ######################################
        collection.update_one(flt, {'$set':bar}, upsert=True)
    print('saveDone')

In [7]:
# transfer2VNPY(btcDf, symbol, collection)

# 定时任务
Timer 函数第一个参数是时间间隔（单位是秒），第二个参数是要调用的函数名，第三个参数是调用函数的参数(tuple)

In [8]:
# from datetime import datetime
# from threading import Timer
# # 打印时间函数
# def printTime(inc):
#     print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
#     t = Timer(inc, printTime, (inc,))
#     t.start()
# # 5s
# printTime(5)

In [9]:
from datetime import datetime
import time
from threading import Timer

cancelTimer = False
def saveNewData(instrument_id, granularity):
    print('saving')
    data = getCandles(instrument_id, granularity)
    symbol = 'btc_usdt_cq.future:okex'
    client = pymongo.MongoClient('localhost', 33333)
    collection = client['VnTrader_1Min_Db_Coin'][symbol]
    collection.create_index([('datetime', pymongo.ASCENDING)], unique=True)
    transfer2VNPY(data, symbol, collection)
    if not cancelTimer:
        t = Timer(65, saveNewData, ('BTC-USDT-200327', 60))
        t.start()
    else:
        print("timerCancel")        

t = Timer(10, saveNewData, ('BTC-USDT-200327', 60))
t.start()

time.sleep(300)
cancelTimer = True

saving
2020-01-10T22:27:47Z 2020-01-11T03:17:47Z 290
2020-01-10T17:37:47Z 2020-01-10T22:27:47Z 290
2020-01-10T12:47:47Z 2020-01-10T17:37:47Z 290
2020-01-10T07:57:47Z 2020-01-10T12:47:47Z 290
2020-01-10T03:07:47Z 2020-01-10T07:57:47Z 280
saveDone
saving
2020-01-10T22:28:54Z 2020-01-11T03:18:54Z 290
2020-01-10T17:38:54Z 2020-01-10T22:28:54Z 290
2020-01-10T12:48:54Z 2020-01-10T17:38:54Z 290
2020-01-10T07:58:54Z 2020-01-10T12:48:54Z 290
2020-01-10T03:08:54Z 2020-01-10T07:58:54Z 280
saveDone
saving
2020-01-10T22:30:01Z 2020-01-11T03:20:01Z 289
2020-01-10T17:40:01Z 2020-01-10T22:30:01Z 290
2020-01-10T12:50:01Z 2020-01-10T17:40:01Z 290
2020-01-10T08:00:01Z 2020-01-10T12:50:01Z 290
2020-01-10T03:10:01Z 2020-01-10T08:00:01Z 281
saveDone
saving
2020-01-10T22:31:08Z 2020-01-11T03:21:08Z 290
2020-01-10T17:41:08Z 2020-01-10T22:31:08Z 290
2020-01-10T12:51:08Z 2020-01-10T17:41:08Z 290
2020-01-10T08:01:08Z 2020-01-10T12:51:08Z 290
2020-01-10T03:11:08Z 2020-01-10T08:01:08Z 281
saveDone
saving
2020-01-1